In [2]:
from PIL import Image
import os, glob

print("PIL 라이브러리 import 완료!")

PIL 라이브러리 import 완료!


In [3]:
import os

def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")

    # 파일마다 모두 28x28 사이즈로 바꾸어 저장합니다.
	target_size=(28,28)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")
	
# 가위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/scissor"
resize_images(image_dir_path)

print("가위 이미지 resize 완료!")


# 바위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/rock"
resize_images(image_dir_path)

print("바위 이미지 resize 완료!")


# 보 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/paper"
resize_images(image_dir_path)

print("보 이미지 resize 완료!")

300  images to be resized.
300  images resized.
가위 이미지 resize 완료!
300  images to be resized.
300  images resized.
바위 이미지 resize 완료!
300  images to be resized.
300  images resized.
보 이미지 resize 완료!


In [4]:
import numpy as np

def load_data(img_path, number_of_data=900):  # 가위바위보 이미지 개수 총합에 주의하세요.
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=28
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는", idx,"입니다.")
    return imgs, labels

image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper"
(x_train, y_train)=load_data(image_dir_path)
x_train_norm = x_train/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))

학습데이터(x_train)의 이미지 개수는 900 입니다.
x_train shape: (900, 28, 28, 3)
y_train shape: (900,)


In [5]:
import tensorflow as tf
from tensorflow import keras
import numpy as np


model = keras.models.Sequential()
model.add(keras.layers.Conv2D(256, (3, 3), activation = 'relu', input_shape=(28, 28, 3)))
model.add(keras.layers.MaxPooling2D(2, 2))
model.add(keras.layers.Conv2D(512, (3, 3), activation = 'relu'))
model.add(keras.layers.MaxPooling2D(2, 2))

### 레이어를 하나 추가 했더니 확률이 늘어났는데 더 추가하면 할수록 확률이 낮아져서 1개만 추가했습니다.
model.add(keras.layers.Conv2D(512, (3, 3), activation = 'relu'))
model.add(keras.layers.MaxPooling2D(2, 2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(512, activation = 'relu'))
model.add(keras.layers.Dense(3, activation = 'softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 256)       7168      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 256)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 512)       1180160   
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 512)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 512)         2359808   
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 1, 1, 512)         0         
_________________________________________________________________
flatten (Flatten)            (None, 512)               0

In [6]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train_norm, y_train, epochs = 10)

Epoch 1/10
29/29 [==============================] - 9s 143ms/step - loss: 1.1530 - accuracy: 0.3342
Epoch 2/10
29/29 [==============================] - 1s 22ms/step - loss: 1.0994 - accuracy: 0.3381
Epoch 3/10
29/29 [==============================] - 1s 22ms/step - loss: 1.0977 - accuracy: 0.3599
Epoch 4/10
29/29 [==============================] - 1s 23ms/step - loss: 1.0719 - accuracy: 0.4408
Epoch 5/10
29/29 [==============================] - 1s 22ms/step - loss: 0.8748 - accuracy: 0.5721
Epoch 6/10
29/29 [==============================] - 1s 23ms/step - loss: 0.7514 - accuracy: 0.6612
Epoch 7/10
29/29 [==============================] - 1s 22ms/step - loss: 0.3927 - accuracy: 0.8163
Epoch 8/10
29/29 [==============================] - 1s 23ms/step - loss: 0.2361 - accuracy: 0.9026
Epoch 9/10
29/29 [==============================] - 1s 22ms/step - loss: 0.7065 - accuracy: 0.7662
Epoch 10/10
29/29 [==============================] - 1s 22ms/step - loss: 0.2524 - accuracy: 0.9058


In [7]:
def resize_image(path):
    image_dir_path = os.getenv("HOME") + path
    images = glob.glob(image_dir_path + "/*.jpg")  
    target_size=(28, 28)
    for img in images:
        old_img = Image.open(img)
        new_img = old_img.resize(target_size, Image.ANTIALIAS)
        new_img.save(img, "JPEG")



def load_data(img_path, number_of_data):
    img_size = 28
    color = 3
    
    imgs = np.zeros(number_of_data * img_size * img_size * color, dtype = np.int32).reshape(number_of_data, img_size, img_size, color)
    labels = np.zeros(number_of_data, dtype = np.int32)

    idx = 0
    for file in glob.iglob(img_path + '/scissor/*.jpg'):
        img = np.array(Image.open(file), dtype = np.int32)
        imgs[idx, :, :, :] = img
        labels[idx] = 0
        idx = idx + 1

    for file in glob.iglob(img_path + '/rock/*.jpg'):
        img = np.array(Image.open(file), dtype = np.int32)
        imgs[idx, :, :, :] = img
        labels[idx] = 1
        idx = idx + 1       
    
    for file in glob.iglob(img_path + '/paper/*.jpg'):
        img = np.array(Image.open(file), dtype = np.int32)
        imgs[idx, :, :, :] = img
        labels[idx] = 2
        idx = idx + 1
        
    return imgs, labels

image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test"
(x_test, y_test)=load_data(image_dir_path, 3000)

x_test_norm = x_test/255.0

resize_image("/aiffel/rock_scissor_paper/test/scissor")
resize_image("/aiffel/rock_scissor_paperr/test/rock")
resize_image("/aiffel/rock_scissor_paperr/test/paper")


In [9]:
test_loss, test_accuracy = model.evaluate(x_test_norm, y_test, verbose = 2)

94/94 - 1s - loss: 0.5746 - accuracy: 0.9223


In [ ]:
'''
회고

처음 데이터 세팅은 미리 설명해준 코드로 세팅했습니다.

그리고 네트워크 설계를 할때, 앞에서 했던 흑백 숫자를 구분할때 변수를 16으로 했었는데, 
가위바위보는 컬러 사진에 조명도 있고 배경도 있다보니 숫자를 256, 512로 높였더니 정확도가 많이 상승했습니다.

그리고 레이어를 하나 더 추가를 했는데 정확도가 또 상승했습니다.
하지만 레이어를 무작정 많이 추가하면 오히려 정확도가 낮아져서 레이어의 의미를 찾아보았더니
데이터에 따라 적정 레이어가 있다는 것을 알았습니다.
그래서 레이어는 3개로 했습니다.

또, 학습을 여러번 초기화하고 학습시켜보았더니 평균적으로 12회를 넘어가면 accuracy는 올라가지만 loss가 너무 많이 올라가 오버피팅이 되어
오히려 정확도가 많이 떨어져서 10회로 학습 횟수를 제한했습니다.
'''
